In [1]:
import torch, torch.nn as nn
import timm
import torch.autograd.profiler as profiler
from einops import reduce, rearrange, repeat
import numpy as np

In [2]:
from triangle_cornermap_segment import TrianglePatchSegment, SimplePatchCornerModule

In [3]:
data = torch.randn(1,3,64,64)

In [ ]:
mdl = timm.create_model("resnet18", pretrained=True, num_classes=0, features_only=True, global_pool='', out_indices=(1,))
%timeit mdl(data)

In [ ]:
for name, mod in mdl.named_modules():
    if ininstance

In [4]:
from tiny_selfattention_model import SelfAttentionModel
d = torch.randn(1,3,64,64)
m = SelfAttentionModel()
%timeit m(d)

16.6 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
coremodel = TrianglePatchSegment(backbone='resnet18')
coremodel.eval()
%timeit coremodel(data)

In [ ]:
coremodel_nostride = TrianglePatchSegment(backbone='resnet18', set_stride_to1=True, outlevel=1)
# coremodel_nostride.feature_extractor.feature_info.channels()
%timeit coremodel_nostride(data)

In [ ]:
from tiny_selfattention_model import ConvBlock
c = ConvBlock(3, 64, 8)
%timeit c(data)

In [ ]:
out = c(data)
out.shape

In [ ]:
q = torch.randn(64,64)
k = torch.randn(64,64)
v = torch.randn(64,64)
%timeit val = self_attention(q,k,v)


In [ ]:
from attrdict import AttrDict
args = AttrDict()
args.backbone = 'tv_resnet34'
# lightmod = SimplePatchCornerModule(args)
# lightmod = SimplePatchCornerModule.load_from_checkpoint('D:/re-search/vod_triangles/src/Colab_logs/checkpoints/epoch=2-step=15000.ckpt', args)
# lightmod = lightmod.eval()

In [ ]:
model = SimplePatchCornerModule(args)
# model.load_state_dict(x['state_dict'])
%timeit model(data)

In [ ]:
coremodel = TrianglePatchSegment(backbone='tv_resnet34')
coremodel.eval()
# %timeit coremodel(data)
# coremodel.double()
# data = data.double()

%timeit coremodel(data)

with profiler.profile(with_stack=True, profile_memory=True) as scprof:
    out = coremodel(data)

# print(scprof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

In [ ]:
device = torch.device('cpu')
x = torch.load('D:/re-search/vod_triangles/src/Colab_logs/checkpoints/epoch=2-step=12000.ckpt', map_location=device)
print(list(x.keys()))

In [ ]:
sd = x['state_dict']
new_sd = {}

ckptvals = []
for k, v in sd.items():
    if 'batches' not in k and 'bce_loss' not in k:
#         if 'bn' in k:
#         v[torch.logical_and(v > -1e-16, v < 1e-16)] = 0.
#         new_sd[k[6:]] = v

        if torch.any(torch.logical_and(v > -1e-16, v < 1e-16)):
            val = v[torch.logical_and(v > -1e-16, v < 1e-16)]
#             print(k, val[:5])

In [ ]:
data = torch.randn(1,3,64,64)
len(new_sd)

In [ ]:
coremodel.load_state_dict(new_sd, strict=False)
coremodel.eval()

%timeit coremodel(data)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out = coremodel(data)

# print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

In [ ]:
%timeit model(data)

In [ ]:
from mobile_unet import MobileUNet
import torch
x = torch.rand((1, 3, 64, 64))

model = MobileUNet()
%timeit out = model(x)

In [ ]:
model

In [5]:
class NonMaxSuppression(nn.Module):
    def __init__(self, n_peaks=256):
        super().__init__()
        self.k = 3  # kernel
        self.p = 1  # padding
        self.s = 1  # stride
        self.center_idx = self.k**2//2
        self.sigmoid = nn.Sigmoid()
        self.unfold = nn.Unfold(kernel_size=self.k, padding=self.p, stride=self.s)
        self.n_peaks = n_peaks

    def sample_peaks(self, x):
        B, _, H, W = x.shape
        for b in range(B):
            x_b = x[b, 0]
            idx = torch.topk(x_b.flatten(), self.n_peaks).indices
            idx_i = torch.div(idx, W, rounding_mode='floor')
            idx_j = idx % W
            idx = torch.cat((idx_i.unsqueeze(1), idx_j.unsqueeze(1)), dim=1)
            idx = idx.unsqueeze(0)

            if b == 0:
                graph = idx
            else:
                graph = torch.cat((graph, idx), dim=0)

        return graph

    def forward(self, feat):
        B, C, H, W = feat.shape

        x = self.sigmoid(feat)

        # Prepare filter
        f = self.unfold(x).view(B, self.k**2, H, W)
        f = torch.argmax(f, dim=1).unsqueeze(1)
        f = (f == self.center_idx).float()

        # Apply filter
        x = x * f

        # Sample top peaks
        graph = self.sample_peaks(x)
        return x, graph

In [6]:
nms = NonMaxSuppression()
mask = torch.randn(1,1,64,64)
%timeit nms(mask)

1.96 ms ± 62.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
x, graph = nms(mask)

In [8]:
x.shape

torch.Size([1, 1, 64, 64])

In [9]:
graph.shape

torch.Size([1, 256, 2])